Dou et al. (2020) model

Agora em Julia versão 1.5.3

This script simulates the model

Calcula momentos a partir de parâmetros arbitrários a fim de testar o algoritmo de minimização.

In [1]:
using LinearAlgebra, Statistics

using DataFrames
using BenchmarkTools


In [2]:
using Distributed
nprocs = 2
addprocs(nprocs);

@everywhere using NBInclude
@everywhere @nbinclude("solve_model.ipynb")

#importou a função solve_tree

pids = workers()
aux = pids[nprocs]

3

In [3]:
#Vh/D, L/D, Dj/D
data1 = [0.86, 0.19, 0.8];
n1 = 49

data2 = [2.65, 0.69, 0.74];
n2 = 17

#ρ, β, c0, λj
game_parameters = [0.7, 9.0, 0.1, 0.2];

In [4]:
A2 = @spawnat aux solve_tree(data2, game_parameters)

A1 = solve_tree(data1, game_parameters);

A2 = fetch(A2);

132.274148 seconds (171.78 M allocations: 3.856 GiB      From worker 3:	113.009988 seconds (142.85 M allocations: 3.216 GiB, 0.99% gc time)
, 1.02% gc time)


Simulation parameters

In [5]:
#θs0, θj0, μ
simulation_parameters = [10.0, 10.0, 6.0];

### Criando funções para simular o jogo

In [159]:
function simulate_moments(U, S, N, game_parameters, simulation_parameters, data1, data2, A1, A2, n1)
    

    Results = loop_simulations(U, S, N, game_parameters, simulation_parameters, data1, data2, A1, A2, n1)

    Moments = zeros(S, 8)

    μ = simulation_parameters[3]
    for s in 1:S

        DF = DataFrame()
        DF.payoff_s = Results[s,:,1];
        DF.payoff_j = Results[s,:,2];
        DF.out = Results[s,:,3];
        DF.t = Results[s,:,4];
        DF.observed_proposals = Results[s,:,5];
        DF.Ds = Results[s,:,6];
        DF.Dj = Results[s,:,7];


        DF.outcome = ifelse.(DF.out .==3.0, "R", "L");


        #desfazendo o deslocamento no índice de t
        DF.t = DF.t .- 1.0;

        # DF.incourt = ifelse.(DF.t .> 0.0, "incourt", "precourt");

        INCOURT = filter(DF -> DF.t .> 0.0, DF);
        PRECOURT = filter(DF -> DF.t .<= 0.0, DF);

        REORG = filter(DF -> DF.outcome .== "R", DF);#casos reorganizados dentro e fora da corte

        PRECOURT_REORG = filter(PRECOURT -> PRECOURT.outcome .== "R", PRECOURT)
        INCOURT_REORG = filter(INCOURT -> INCOURT.outcome .== "R", INCOURT)

        #cálculo dos momentos
        #1. avg log number of months between observed proposals incourt

        mm1 = INCOURT
        mm1 = log.(mm1.t .* μ ./ mm1.observed_proposals)
        mm1 = mean(mm1)

        #2. fraction reorganized given that the case went into court

        mm2 = INCOURT
        mm2 = size(filter(mm2 -> mm2.outcome .== "R", mm2),1)/ size(mm2,1)

        #3. ln duration of court cases in months

        mm3 = INCOURT
        mm3 = filter(mm3 -> mm3.t .> 0.0, mm3) #removendo os casos 0 para não poluir a média
        mm3.t = mm3.t .* μ
        mm3 = mean(log.(mm3.t)) #log here uses exp as base, so it's the same as ln

        #4. fraction of cases incourt
        mm4 = size(INCOURT,1) / size(DF, 1)

        #5. avg recovery rate for S given a REORGANIZATION in the 25% fastest cases
        q = 0.25 #quartil desejado

        mm5 = REORG

        if( size(mm5, 1) == 0) #se não tiver casos de reorganização dentro ou fora da corte, retorna zero
            mm5 = 0.0
            mm6 = 0.0
        else

            mm5.t = mm5.t .* μ #total duration
            quartil = quantile(log.(mm5.t), q)

            mm5 = filter(mm5 -> log.(mm5.t) .<= quartil, mm5)
            mm5.R_s = mm5.payoff_s ./ mm5.Ds
            mm5 = mean(mm5.R_s)

            #6. avg recovery rate for J given a REORGANIZATION in the 25% fastest cases
            mm6 = REORG
            mm6 = filter(mm6 -> log.(mm6.t) .<= quartil, mm6)
            mm6.R_j = mm6.payoff_j ./ mm6.Dj
            mm6 = mean(mm6.R_j)

        end


        #7. junior avg fraction gain given incourt REORGANIZATION

        mm7 = INCOURT_REORG
        mm7 = mean( (mm7.payoff_j ./ (mm7.payoff_j .+ mm7.payoff_s ))  )

        #8. total recovery rate given incourt REORGANIZATION

        mm8 = INCOURT_REORG
        mm8 = mean( mm8.payoff_s .+ mm8.payoff_j)

        Moments[s,:] .= [mm1, mm2, mm3, mm4, mm5, mm6, mm7, mm8]


        #if there are no INCOURT or no PRECOURT cases, it will return NaN
        #thus, we replace it by zero
        Moments[s,:] .= replace!(Moments[s,:], NaN=> zero(0.0))

    end





    #9. slope of the regression of log(recovery rate | incourt reorg) ~ log(duration)

    #fazendo a regressão do nono momento fora do loop para poupar tempo
    #isto é, ao invés de fazer uma regressão para cada simulação,
    #faremos uma regressão só usando todas as simulações
    df = DataFrame()

    for s in 1:S
        df_result = DataFrame(Results[s,:, 1:4])
        append!(df, df_result)
    end

    rename!(df, [:x1, :x2, :x3, :x4] .=> [:payoff_s, :payoff_j, :out, :t])



    #desfazendo o deslocamento no índice de t
    df.t = df.t .- 1.0;

    #keeping only incourt reorganizations
    df = filter(df -> df.t .> 0.0, df);
    df = filter(df -> df.out .== 3.0, df)


    x = log.(df.payoff_s .+ df.payoff_j)

    X = [ones(size(x,1)) x]

    y = log.(df.t .* μ)

    if(size(df, 1) == 0 || rank(X' * X) != size(X,2))
        #the second condition is to make sure that (X' * X) is singular before calculating mm9
        mm9 = zero(0.0)
    else
        mm9 = (X' * X) \ (X' * y)
        mm9 = mm9[2]
    end


    MM = vec(mean(Moments, dims=1))
    MM = vcat(MM, mm9)

    return MM

end
    
    

function loop_simulations(U, S, N, game_parameters, simulation_parameters, data1, data2, A1, A2, n1)


    #U is an array if dimensions S x N x 120 of fixed draws of the uniform distribution
        #120 is the upper bound of draws needed: 4 for each round (with cram dowm), maximum of 30 rounds
    #S is the number of simulations
    #N is the number of observations

    #data1 is the center of the first cluster
    #A1 is the collection of arrays from the first cluster (s_W, j_W, Pst, Pjt)
    #n1 is the number of observations in the first cluster



    #no futuro os argumentos podem ser arrays de arrays, assim ele faz o loop para cada cluster
    Results = zeros(S, N, 7)

    #últimas entradas de Results são os valores das dívidas


    for s in 1:S
        for n in 1:N
            if(n <= n1)
                Results[s,n,1:5] .= simulate_game(U[s,n,:], 1, data1, game_parameters, simulation_parameters, A1[1], A1[2], A1[3], A1[4])
                Results[s,n, end-1] = A1[5]
                Results[s,n, end] = A1[6]
            else
                Results[s,n,1:5] .= simulate_game(U[s,n,:], 1, data2, game_parameters, simulation_parameters, A2[1], A2[2], A2[3], A2[4])
                Results[s,n, end-1] = A2[5]
                Results[s,n, end] = A2[6]
            end

        end
    end

    return Results

end


function simulate_game(U_sn, draw, data, game_parameters, simulation_parameters, s_W=s_W, j_W=j_W, Pst_array=Pst_array, Pjt_array=Pjt_array, t=1, grid=100)


    #U_sn is a vector of U made for simulation s of observation n

#         Vmax = data[1]
#         L = data[2]
#         Dj = data[3]

    Vmax, L, Dj = data
    Ds = one(1.0) - Dj



    ρ, β, c0, λj = game_parameters;
    c1 = c0/30;

#         c0 = game_parameters[3];
#         c1 = c0/30;

#         ρ = game_parameters[1];
#         λj = game_parameters[4];


#         draw = 1 #index to indicate which draw it is



#     hst = Hs0
#     hjt = Hj0

    hst = Int64(simulation_parameters[1])
    hjt = Int64(simulation_parameters[2])

    #assumindo que lower bounds nos períodos iniciais são as próprias habilidades iniciais
    lst = hst
    ljt = hjt

    result = zeros(5);

    #number of observed proposals
    observed_proposals = 0.0


    #recovering "T+1" from s_W 
    T = size(s_W,1)




    while(result[1]==0.0 && t < T)


        #3 draws per turn
        u1 = U_sn[draw]
        draw+=1

        u2 = U_sn[draw]
        draw+=1

        u3 = U_sn[draw]
        draw+=1


        if(u1 < λj)
            propositor = "j"

        else
            propositor = "s"

        end


        #setting the default variables according to the propositor
        Pkt_array, Cont_val, prop_index, respondent_index, m_L, hkt, lkt, hmt, lmt, lk_next, hk_next, hm_next = choose_parameters(u2, u3, propositor, hst, lst, hjt, ljt, Pst_array, Pjt_array, s_W, j_W)

        #proposal ####
        policy, payoff_prop, payment, lm_next = proposal(Pkt_array, t, hkt, lmt)

        #lm_next is the update of the adversary's lower bound
        if(lm_next>=grid+1 || policy!= 3.0) #para evitar update de lowerbound quando proponente não propõe reorg
            lm_next = lmt
        else
            lm_next = Int64(lm_next)
        end



        if(policy==3.0)

            observed_proposals += 1.0

            payoff_respondent, answer = answer_reorg(payment, Cont_val, t, hm_next, lm_next, lk_next)


            if(answer==1.0)

                result[prop_index] = payoff_prop
                result[respondent_index] = payoff_respondent
                result[3] = 3.0
                result[4] = t
                result[5] = observed_proposals

            else

                t+=1

                hst, lst, hjt, ljt = update_beliefs(propositor, hk_next, hm_next, lk_next, lm_next)
            end

        elseif(policy==2.0)

            t+=1

            hst, lst, hjt, ljt = update_beliefs(propositor, hk_next, hm_next, lk_next, lm_next)



        else      
            #(policy==1.0)

            observed_proposals += 1.0

            payoff_respondent, answer = answer_liq(m_L, t, hm_next, payoff_prop, L, Ds, Dj, Vmax, ρ, c0, c1)



            if(answer==1.0)

                result[prop_index] = payoff_prop
                result[respondent_index] = payoff_respondent
                result[3] = 1.0
                result[4] = t
                result[5] = observed_proposals

            else

                result[prop_index] = payoff_prop
                result[respondent_index] = payoff_respondent
                result[3] = 3.0
                result[4] = t
                result[5] = observed_proposals


            end
        end


        if(t==T)

            result[1] = s_W[T, 1, 1, 1]#todos os valores finais de S são iguais, então acessarei o índice 1
            result[2] = j_W[T, 1, 1, 1]
            result[3] = 1.0
            result[4] = t
            result[5] = max(observed_proposals, 1.0)

        end





    end


    return result
end


function choose_parameters(u2, u3, propositor, hst, lst, hjt, ljt, Pst_array, Pjt_array, s_W, j_W)

    if(propositor=="s")

        Pkt_array = Pst_array
        Cont_val = j_W
        prop_index = 1
        respondent_index = 2 #índice de j, para organizar o payoff
        m_L = j_L

        hkt = hst
        lkt = lst

        hmt = hjt
        lmt = ljt


        lk_next = hkt


        hk_next = draw_beta(u2, hkt)
        hm_next = draw_beta(u3, hmt)

    else
#             propositor=="j"
        Pkt_array = Pjt_array
        Cont_val = s_W
        prop_index = 2
        respondent_index = 1
        m_L = s_L

        hkt = hjt
        lkt = ljt

        hmt = hst
        lmt = lst


        lk_next = hkt


        hk_next = draw_beta(u2, hkt)
        hm_next = draw_beta(u3, hmt)
    end

    return Pkt_array, Cont_val, prop_index, respondent_index, m_L, hkt, lkt, hmt, lmt, lk_next, hk_next, hm_next
end


function draw_beta(u, hkt, β=game_parameters[2], grid=100)


    if(hkt == grid)

        return grid

    else

        x = 1.0 - exp(1.0/β * (log(1.0 - u) + β * log(1.0 - hkt/grid)))
        x = round(x * grid, digits=0)
#         return Int64(x * 100) #to convert in an integer
        return Int64(x)

    end
end


#cost function
function Ct(t, c0=c0, c1=c1)

    #didn't use D because is normalized to D == 1.0
    #cost at period t=0(index1) is 0
    if(t <= 1)
        return 0
    else
        return c0 + c1 * (t-1) #test to make index==1 be t==0
    end
end



#liquidation payoffs
function s_L(t, L, Ds, Dj, c0, c1)
    return min(L - Ct(t, c0, c1), Ds)
end


function j_L(t, L, Ds, Dj, c0, c1)
    return min(L - Ct(t, c0, c1) - s_L(t, L, Ds, Dj, c0, c1), Dj)
end


function proposal(Pkt_array, t, hkt, lmt)

    #pkt array será sempre do propositor, quem responder às propostas apenas olhará o seu valor de continuação

    return policy, payoff_prop, payment, lm_next = Pkt_array[t, hkt, lmt, [end,end-1, 1, 2]]

end


#maximum value of reorganization each period
function Vt(Vmax, ρ, t)

    if(t <=1)
        return Vmax
    else
        #(t-2) instead of (t-1) because we shifted the indexes in the game so as to include t==0 at index==1
        return ρ^(t-2) * Vmax
    end


end

# answer_liq
function answer_liq(m_L, t, hm_next, payoff_prop, L, Ds, Dj, Vmax, ρ, c0, c1)

    liq = (m_L(t, L, Ds, Dj, c0, c1), Vt(Vmax, ρ, t) * hm_next/100 - Ct(t, c0, c1) - payoff_prop)

    payoff_liq, answer = findmax(liq)

    return payoff_liq, answer
end

function answer_reorg(payment, Cont_val, t, hm_next, lm_next, lk_next)

    reorg_value = (payment, Cont_val[t+1, hm_next, lm_next, lk_next])

    payoff_reorg, answer = findmax(reorg_value)

    return payoff_reorg, answer
end

function update_beliefs(propositor, hk_next, hm_next, lk_next, lm_next)

    if(propositor=="s")
        hst = hk_next
        lst = lk_next

        hjt = hm_next
        ljt = lm_next

    else
        hst = hm_next
        lst = lm_next

        hjt = hk_next
        ljt = lk_next
    end


    return hst, lst, hjt, ljt
end

update_beliefs (generic function with 1 method)

In [160]:
#unit testing
S = 100
N = 66

using DelimitedFiles

U = readdlm("U.csv", ',')
U = reshape(U, (S, N, 120));

In [161]:
#unit testing
s = 1
n = 1
@btime simulate_game(U[s,n,:], 1, data1, game_parameters, simulation_parameters, A1[1], A1[2], A1[3], A1[4])

  10.000 μs (56 allocations: 3.05 KiB)


5-element Array{Float64,1}:
 0.10211691916287892
 0.0006189333333333332
 3.0
 3.0
 1.0

In [162]:
data2

3-element Array{Float64,1}:
 2.65
 0.69
 0.74

In [163]:
Results = loop_simulations(U, S, N, game_parameters, simulation_parameters, data1, data2, A1, A2, n1)

100×66×7 Array{Float64,3}:
[:, :, 1] =
 0.102117   0.122221   0.0972567  0.0833333  …  0.418     0.26      0.26
 0.19       0.0766667  0.169208   0.19          0.6565    0.938843  0.26
 0.19       0.19       0.19       0.19          0.260152  0.851623  0.26
 0.19       0.19       0.0833333  0.19          0.26      0.524502  0.36351
 0.19       0.19       0.19       0.0948184     0.421034  0.2855    0.26
 0.0833333  0.19       0.19       0.094573   …  0.260152  0.26      0.26
 0.19       0.19       0.19       0.19          0.577     0.26      0.260052
 0.122221   0.136576   0.19       0.19          0.26      0.26      0.26
 0.0833333  0.19       0.19       0.19          0.5717    0.26      0.26
 0.19       0.19       0.19       0.19          0.26      0.26      0.26
 0.0905468  0.19       0.19       0.19       …  0.26      0.26      0.562097
 0.19       0.19       0.19       0.0833333     0.26      0.26      0.26
 0.111732   0.19       0.19       0.0833333     0.26      0.26      0.4594

In [164]:
div = Results[s, :, 7]
size(filter(div -> div .== 0.8, div))

(49,)

In [173]:
μ = simulation_parameters[3]

DF = DataFrame()
DF.payoff_s = Results[s,:,1];
DF.payoff_j = Results[s,:,2];
DF.out = Results[s,:,3];
DF.t = Results[s,:,4];
DF.observed_proposals = Results[s,:,5];
DF.Ds = Results[s,:,6];
DF.Dj = Results[s,:,7];

In [174]:
findmax(DF.payoff_s)

(0.51605, 52)

In [175]:
DF[52, :]

,payoff_s,payoff_j,out,t,observed_proposals,Ds,Dj
,Float64,Float64,Float64,Float64,Float64,Float64,Float64
52,0.51605,0.323333,3.0,3.0,1.0,0.26,0.74


In [176]:
#note que n = 60 > n1 = 17
simulate_game(U[s, 52,:], 1, data2, game_parameters, simulation_parameters, A2[1], A2[2], A2[3], A2[4])

5-element Array{Float64,1}:
 0.51605
 0.32333333333333325
 3.0
 3.0
 1.0

In [177]:
#note que n = 60 > n1 = 17
simulate_game(U[s,66,:], 1, data2, game_parameters, simulation_parameters, A2[1], A2[2], A2[3], A2[4])

5-element Array{Float64,1}:
 0.26
 0.31999999999999995
 1.0
 4.0
 1.0

In [171]:
simulated_moments = simulate_moments(U, S, N, game_parameters, simulation_parameters, data1, data2, A1, A2, n1)

9-element Array{Float64,1}:
  2.2970684309056457
  0.6432371399477048
  2.429059230223651
  0.35469696969696973
  0.8164056029289886
  0.09586820371994731
  0.3210073191433458
  0.44908622910889695
 -0.14478619372051074

## Adaptar o código daqui para baixo

Calculando a derivada dos momentos em relação aos parâmetros

In [28]:
function simulate_new_moments(i, S, N, game_parameters, simulation_parameters, data1, data2, A1, A2, n1)
    
    
    println("parameter ",i, game_parameters, simulation_parameters)
        
    θs0, θj0, μ = simulation_parameters;
    θs0 = Int64(round(θs0, digits=0));
    θj0 = Int64(round(θj0, digits=0));

    λj = game_parameters[4];
    new_simulated_moments = simulate_moments(S, N, game_parameters, simulation_parameters, data1, data2, A1, A2, n1) 
    
    return new_simulated_moments
end

simulate_new_moments (generic function with 1 method)

In [29]:
 
function derivative_moments(S, N, game_parameters, simulation_parameters, data1, data2, n1, aux)

    
    
    
    #https://en.wikibooks.org/wiki/Introduction_to_Numerical_Methods/Numerical_Differentiation
    #backward divided difference method

   


    estimated_game_parameters = game_parameters
    estimated_simulation_parameters = simulation_parameters



    A2 = @spawnat aux solve_tree(data2, game_parameters)
    A1 = solve_tree(data1, game_parameters)
    A2 = fetch(A2)
    
    
    
    simulated_moments = simulate_moments(S, N, game_parameters, simulation_parameters, data1, data2, A1, A2, n1)

    P = size(game_parameters,1)+size(simulation_parameters,1)
    M = size(simulated_moments,1)
    D = zeros(M, P)
    
    #porcentagem de variação no parâmetro
    pct = 1/100
    
    for p in P:-1:1

        if(p>size(game_parameters,1))

            simulation_parameters = copy(estimated_simulation_parameters)
            
            delta = simulation_parameters[p-size(game_parameters,1)] * pct

            simulation_parameters[p-size(game_parameters,1)] -= delta


            if(p == 5 || p ==6) #para ajustar caso seja habilidade inicial de S ou de J
                simulation_parameters[p-size(game_parameters,1)] = round(simulation_parameters[p-size(game_parameters,1)], digits=0)
            end
            
            
            #caso a variação percentual não seja suficiente para reduzir habilidades iniciais, delta será 1
            if(simulation_parameters[p-size(game_parameters,1)] == estimated_simulation_parameters[p-size(game_parameters,1)])
                delta = Int64(1)
                
                simulation_parameters[p-size(game_parameters,1)] -= delta
            end

            new_simulated_moments = simulate_new_moments(p, S, N, game_parameters, simulation_parameters, data1, data2, A1, A2, n1)

            D[:, p] = (simulated_moments - new_simulated_moments) ./ delta
            
        else

            #se for um parâmetro de jogo, precisa recalcular o jogo

            game_parameters = copy(estimated_game_parameters)
            
            delta = game_parameters[p] * pct
            game_parameters[p] -= delta

            A2 = @spawnat aux solve_tree(data2, game_parameters)
            A1 = solve_tree(data1, game_parameters)
            A2 = fetch(A2)



            new_simulated_moments = simulate_new_moments(p, S, N, game_parameters, simulation_parameters, data1, data2, A1, A2, n1)

            D[:, p] = (simulated_moments - new_simulated_moments) ./ delta


        end


    end
    
    
    
    return D
end
    

derivative_moments (generic function with 1 method)

In [30]:
Derivative = derivative_moments(S, N, game_parameters, simulation_parameters, data1, data2, n1, aux)

 47.812134 seconds (141.59 M allocations: 3.154 GiB, 2.63% gc time)
      From worker 3:	 36.023834 seconds (141.59 M allocations: 3.154 GiB, 1.82% gc time)
parameter 7[0.639, 8.553, 0.01, 0.011][11.0, 90.0, 4.4975700000000005]
parameter 6[0.639, 8.553, 0.01, 0.011][11.0, 89.0, 4.543]
parameter 5[0.639, 8.553, 0.01, 0.011][10.0, 90.0, 4.543]
       From worker 3:	 53.414062 seconds (141.59 M allocations: 3.154 GiB, 18.75% gc time)
71.382463 seconds (141.59 M allocations: 3.154 GiB, 33.65% gc time)
parameter 4[0.639, 8.553, 0.01, 0.010889999999999999][10.0, 90.0, 4.543]
       From worker 3:	 43.159837 seconds (141.59 M allocations: 3.154 GiB, 1.75% gc time)
48.536867 seconds (141.59 M allocations: 3.154 GiB, 1.18% gc time)
parameter 3[0.639, 8.553, 0.0099, 0.011][10.0, 90.0, 4.543]
       From worker 3:	 42.902130 seconds (141.59 M allocations: 3.154 GiB, 1.16% gc time)
49.739593 seconds (141.59 M allocations: 3.154 GiB, 1.44% gc time)
parameter 2[0.639, 8.46747, 0.01, 0.011][10.0, 90.

8×7 Array{Float64,2}:
   1.30515    -0.271256    -1298.28    -1289.38   …  -0.00579645   0.0908207
  -2.29123     0.246008     3795.15     3638.48       0.0114915    0.133209
   1.32089    -0.335136     -924.149    -946.019     -0.012412     0.113216
   0.331958    0.00708594   7351.52     6673.55       0.00521886  -0.00667027
 -14.7515     -0.314492    -3540.79    -3201.93      -0.172943     1.62298
 -11.1003     -0.214215    -1867.52    -1673.07   …  -0.131371     1.79867
  -0.0167323  -0.0942029      65.6934    158.822      0.0013325   -0.065914
   0.0712664   0.0869639   -4248.81    -3818.33       0.0122835    0.088857

In [31]:
Derivative[:,2]

8-element Array{Float64,1}:
 -0.27125629181626487
  0.24600758115314386
 -0.33513560482442567
  0.00708594184567402
 -0.31449220582857806
 -0.21421507792458255
 -0.09420287591189372
  0.08696391795142722

In [32]:
using DelimitedFiles

writedlm( "Derivative-nocd90_1pct.csv",  Derivative, ',')